In [1]:
!pip -q install -q git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install -q  bitsandbytes accelerate xformers einops
!pip -q install langchain


[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

torch.set_default_device('cuda')

model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-alpha",
                                             torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha",
                                          torch_dtype="auto")


pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant for coding and programming questions.",
    },
    {"role": "user", "content": "Can you explain the concept of recursion in programming?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1471: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a helpful assistant for coding and programming questions.</s>
<|user|>
Can you explain the concept of recursion in programming?</s>
<|assistant|>
Sure! Recursion is a programming concept that allows a function to call itself repeatedly until a certain condition is met. The word "recursion" itself comes from the Latin word "recursus," which means "to run back."

In simpler terms, recursion is a way for a function to solve a problem by breaking it down into smaller and smaller sub-problems until it reaches a base case, which is a point where the function can no longer be called recursively.

Here's an example: let's say you want to calculate the factorial of a number using recursion. The factorial of a number is the product of that number and all the positive integers smaller than it.

For example, the factorial of 5 is 5 * 4 * 3 * 2 * 1, which equals 120.

To calculate the factorial of a number using recursion, we write a function that calls itself until we reach a ba

In [3]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [5]:
def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = f"""<|im_start|> system\n{system_prompt}<|im_end|>"""
    else:
        system_prompt = ""
    prompt = f"""<|im_start|> user\n{input_text}<|im_end|>"""
    final_prompt = system_prompt + prompt
    inputs = tokenizer(final_prompt, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             pad_token_id = 3200,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    # text = text[len(final_prompt):] if text.startswith(final_prompt) else text
    text = text.replace(final_prompt, '', 1)
    wrapped_text = wrap_text(text)
    print(wrapped_text)


def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly chatbot who always responds in the style of a pirate"
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": input_text},
    ]

    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)

    outputs = pipe(prompt,
                   max_new_tokens=max_length,
                   do_sample=True,
                   temperature=0.7,
                   top_k=50,
                   top_p=0.95)
    text = outputs[0]["generated_text"]
    text = text.replace(prompt, '', 1)
    wrapped_text = wrap_text(text)
    print(wrapped_text)

In [6]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Here's a possible implementation:

```python
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def print_prime(n):
    for i in range(2, n+1):
        if is_prime(i):
            print(i)
```

The `is_prime` function checks if a number is prime by iterating over all numbers between
2 and the square root of the number. If the number is divisible by any of these numbers,
then it's not prime.

The `print_prime` function uses a for loop to iterate over all numbers between 2 and n,
and checks if each number is prime using the `is_prime` function. If a prime number is
found, it's printed to the console using the `print` function.


In [7]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Mathematics is often referred to as a language that speaks in numbers and symbols. It can
be seen as a beacon, guiding us through complex concepts and ideas. In the same way, a
lighthouse is a beacon that guides ships through dangerous and uncertain waters.

Like a lighthouse, mathematics helps us navigate through the unknown. A lighthouse emits a
steady beam of light that signals to ships in the distance, providing a clear and reliable
path to follow. Similarly, mathematics provides a set of rules and principles that allow
us to navigate through complex problems and arrive at logical and reliable solutions.

Both mathematics and lighthouses require precision and attention to detail. The light of a
lighthouse must be precisely aligned and maintained to ensure that it remains a reliable
guide for ships. Similarly, mathematical calculations and formulas must be precise and
accurate to ensure that they provide reliable results.

In both cases, there is a certain beauty in the precision an